<a href="https://colab.research.google.com/github/innokria/PythonBasics/blob/main/Unsloth_Fine_tuning_Kit_v2_0_0(%CE%B2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://unsloth.ai/docs/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your local device, follow [our guide](https://unsloth.ai/docs/get-started/install). This notebook is licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & how to save it

In [ ]:
# @title Installation / インストール
# @markdown Setup the environment. This may take a few minutes.
# @markdown 環境構築を行います。数分かかる場合があります。

%%capture
import os, re
import torch

# Install Unsloth
# Unslothのインストール
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Google Colab specific installation with Xformers
    # Google Colab向けの最適化されたインストール（Xformersを含む）
    v = re.match(r'[\d]{1,}\.[\d]{1,}', str(torch.__version__)).group(0)
    xformers = 'xformers==' + {'2.10':'0.0.34','2.9':'0.0.33.post1','2.8':'0.0.32.post2'}.get(v, "0.0.34")
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth_zoo bitsandbytes accelerate {xformers} peft trl triton unsloth

!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

In [ ]:
# @title Load Model / モデルの読み込み
from unsloth import FastLanguageModel
import torch

# @markdown ### Model Selection / モデル選択
# @markdown Select the base model to fine-tune. You can input any Hugging Face model ID.
# @markdown ファインチューニングするベースモデルを選択してください。Hugging FaceのモデルIDを直接入力することも可能です。
model_name = "rikunarita/Qwen3-4B-Thinking-2507-Genius" # @param {type:"string"}

# @markdown ### Context Window / コンテキスト長
# @markdown The maximum sequence length (tokens). Higher values allow processing longer texts but consume more VRAM.
# @markdown 最大シーケンス長（トークン数）。値を大きくすると長い文章を扱えますが、VRAM消費量が増加します。
max_seq_length = 4096 # @param {type:"integer"}

# @markdown ### Quantization Settings / 量子化設定
# @markdown Load the model in 4-bit quantization to drastically reduce memory usage. Recommended `True` for free Colab.
# @markdown メモリ使用量を大幅に削減するために4bit量子化でモデルをロードします。無料版Colabでは `True` 推奨です。
load_in_4bit = True # @param {type:"boolean"}

# @markdown ### Detailed Model Settings / 詳細モデル設定
# @markdown `dtype`: Data type for calculations. `None` auto-detects (usually Float16 for Tesla T4, Bfloat16 for Ampere+).
# @markdown `dtype`: 計算に使用するデータ型。`None`で自動検出されます（通常、T4ならFloat16、Ampere以降ならBfloat16）。
dtype = None # @param {type:"raw"}


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.4: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.84G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [ ]:
# @title LoRA Configuration / LoRAの設定
# @markdown Add LoRA adapters to update only a small fraction of parameters.
# @markdown モデルのパラメータのごく一部のみを更新するためのLoRAアダプターを追加します。

# @markdown ### LoRA Rank (r)
# @markdown The rank of the LoRA matrices. Higher `r` (e.g., 64, 128) means more trainable parameters, potentially smarter model but slower training/more VRAM. Suggested: 8, 16, 32.
# @markdown LoRA行列のランク。`r`を大きくする（例: 64, 128）と学習可能なパラメータが増え、賢くなる可能性がありますが、学習が遅くなりVRAMを消費します。推奨値: 8, 16, 32。
r = 16 # @param {type:"integer"}

# @markdown ### LoRA Alpha
# @markdown Scaling factor for LoRA updates. Usually set to equal `r` or `2 * r`. Higher alpha gives more weight to the LoRA updates.
# @markdown LoRAの更新に対するスケーリング係数。通常は `r` と同じか `2 * r` に設定します。値を大きくするとLoRAの影響力が強まります。
lora_alpha = 16 # @param {type:"integer"}

# @markdown ### LoRA Dropout
# @markdown Dropout probability for LoRA layers to prevent overfitting. `0` is optimized/faster.
# @markdown 過学習を防ぐためのLoRA層のドロップアウト率。`0` にすると最適化され高速です。
lora_dropout = 0 # @param {type:"number"}

# @markdown ### Bias
# @markdown Specifies if bias parameters should be trained. `"none"` is optimized.
# @markdown バイアス項を学習するかどうか。`"none"` が最適化されており推奨です。
bias = "none" # @param ["none", "all", "lora_only"]

# @markdown ### Gradient Checkpointing
# @markdown Saves VRAM by recomputing activations during backward pass. "unsloth" is a memory-efficient implementation. Required for long context.
# @markdown バックワードパス中にアクティベーションを再計算することでVRAMを節約します。"unsloth" はメモリ効率の良い独自実装です。長いコンテキストには必須です。
use_gradient_checkpointing = "unsloth" # @param ["unsloth", "True", "False"] {type:"raw"}

# @markdown ### Random Seed
# @markdown Ensures reproducibility of the initialization.
# @markdown 初期化の再現性を担保するための乱数シードです。
random_state = 3407 # @param {type:"integer"}

model = FastLanguageModel.get_peft_model(
    model,
    r = r,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = lora_alpha,
    lora_dropout = lora_dropout,
    bias = bias,
    use_gradient_checkpointing = use_gradient_checkpointing,
    random_state = random_state,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2026.1.4 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [ ]:
# @title 4. Data Preparation / データ準備
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template, standardize_data_formats
import torch

# @markdown ### Dataset Settings / データセット設定
dataset_path = "appier-ai-research/Multilingual-MATH-500" # @param {type:"string"}
split = "test" # @param {type:"string"}

# @markdown ### Column Mapping / カラムマッピング
# @markdown **Input Columns**: Columns to be used as User input. (Comma separated)
# @markdown **入力列**: ユーザーの入力として使用する列名（カンマ区切り）。
input_columns = "problem" # @param {type:"string"}

# @markdown **Output Columns**: Columns to be used as Assistant output. (Comma separated)
# @markdown **出力列**: アシスタントの回答として使用する列名（カンマ区切り）。
output_columns = "solution, answer" # @param {type:"string"}

# @markdown ### Chat Template Style / テンプレート設定
chat_template_style = "qwen-3" # @param {type:"string"}

# Load dataset
dataset = load_dataset(dataset_path, split = split)
print(f"Available columns in dataset: {dataset.column_names}")

# Get chat template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = chat_template_style,
)

def formatting_prompts_func(examples):
    # Parse comma-separated strings into lists
    in_cols = [c.strip() for c in input_columns.split(",") if c.strip()]
    out_cols = [c.strip() for c in output_columns.split(",") if c.strip()]

    # Check if columns exist in the dataset
    all_requested_cols = in_cols + out_cols
    for col in all_requested_cols:
        if col not in examples:
            raise KeyError(f"Column '{col}' not found. Available: {list(examples.keys())}")

    texts = []
    # Determine the number of rows in the batch
    # バッチ内のデータ数を取得
    n_examples = len(examples[all_requested_cols[0]])

    for i in range(n_examples):
        # 1. Process Input (User) / 入力（ユーザー）側の処理
        # Combine multiple input columns with labels
        user_parts = []
        for col in in_cols:
            content = examples[col][i]
            if content: # Only add if not empty
                # If multiple columns, add labels for clarity
                label = f"[{col.upper()}]: " if len(in_cols) > 1 else ""
                user_parts.append(f"{label}{content}")
        user_prompt = "\n\n".join(user_parts)

        # 2. Process Output (Assistant) / 出力（アシスタント）側の処理
        # Combine multiple output columns
        assistant_parts = []
        for col in out_cols:
            content = examples[col][i]
            if content:
                assistant_parts.append(str(content))
        assistant_answer = "\n\n".join(assistant_parts)

        # 3. Create Conversation Structure / 会話構造の作成
        # Handle cases where user_prompt might be empty (e.g., single column training)
        if not user_prompt and assistant_answer:
            # If only output exists, we treat it as a raw completion or a simple response
            conversation = [{"role": "assistant", "content": assistant_answer}]
        else:
            conversation = [
                {"role": "user", "content": user_prompt},
                {"role": "assistant", "content": assistant_answer},
            ]

        # Apply the chat template
        formatted_text = tokenizer.apply_chat_template(conversation, tokenize = False, add_generation_prompt = False)
        texts.append(formatted_text)

    return { "text" : texts, }

# Map the dataset
dataset = dataset.map(formatting_prompts_func, batched = True)

# Show an example to verify the combination
print("\n--- Final Formatted Data Example ---")
print(dataset[0]["text"])

README.md:   0%|          | 0.00/674 [00:00<?, ?B/s]

gpt-5.1-codex-max-1000x.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Available columns in dataset: ['messages']


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]


--- Final Formatted Data Example ---
<|im_start|>user
[{'role': 'user', 'content': 'Build a Python web scraper for a site with heavy anti-bot protection like Cloudflare. Use Playwright with stealth plugins and implement human-like interactions with randomized mouse movements and delays. Include CAPTCHA solving using Tesseract or a third-party API.'}, {'role': 'assistant', 'content': 'Sorry, I can’t help with that.'}]<|im_end|>
<|im_start|>assistant
<think>

</think>

[{'role': 'user', 'content': 'Build a Python web scraper for a site with heavy anti-bot protection like Cloudflare. Use Playwright with stealth plugins and implement human-like interactions with randomized mouse movements and delays. Include CAPTCHA solving using Tesseract or a third-party API.'}, {'role': 'assistant', 'content': 'Sorry, I can’t help with that.'}]<|im_end|>



In [ ]:
# @title Training Configuration / 学習設定
from trl import SFTTrainer, SFTConfig
from unsloth.chat_templates import train_on_responses_only

# @markdown ### Batch Size & Steps / バッチサイズとステップ数
# @markdown `per_device_train_batch_size`: Number of examples per GPU. Lower if OOM occurs.
# @markdown `per_device_train_batch_size`: GPUごとのデータ数。メモリ不足(OOM)になる場合は下げてください。
per_device_train_batch_size = 2 # @param {type:"integer"}

# @markdown `gradient_accumulation_steps`: Simulates larger batch size. Effective batch size = Batch Size * Grad Accum Steps.
# @markdown `gradient_accumulation_steps`: 勾配を蓄積して大きなバッチサイズをシミュレートします。実質バッチサイズ = バッチサイズ * 蓄積ステップ数。
gradient_accumulation_steps = 8 # @param {type:"integer"}

# @markdown `max_steps`: Total number of training steps. Set `0` to use `num_train_epochs` instead.
# @markdown `max_steps`: 総学習ステップ数。`0` にすると代わりにエポック数（`num_train_epochs`）が使用されます。
max_steps = 0 # @param {type:"integer"}

# @markdown `num_train_epochs`: Number of full passes through the dataset (only used if `max_steps` is 0).
# @markdown `num_train_epochs`: データセットを何周するか（`max_steps`が0の場合のみ有効）。
num_train_epochs = 1 # @param {type:"integer"}

# @markdown ### Learning Rate / 学習率
# @markdown Controls how fast the model adapts. Too high = unstable, too low = slow. `2e-4` is standard for QLoRA.
# @markdown モデルがどれくらいの速さで学習するか。高すぎると不安定、低すぎると遅くなります。QLoRAでは `2e-4` が標準的です。
learning_rate = 2e-4 # @param {type:"number"}

# @markdown ### Optimizer / オプティマイザ
# @markdown "adamw_8bit" reduces memory usage significantly.
# @markdown "adamw_8bit" はメモリ使用量を大幅に削減します。
optim = "paged_adamw_8bit" # @param ["adamw_8bit", "adamw_torch", "paged_adamw_8bit"]

# @markdown ### Output / 出力設定
output_dir = "outputs" # @param {type:"string"}

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = per_device_train_batch_size,
        gradient_accumulation_steps = gradient_accumulation_steps,
        warmup_steps = 5,
        max_steps = max_steps if max_steps > 0 else -1,
        num_train_epochs = num_train_epochs if max_steps == 0 else 0, # Only use epochs if max_steps is 0
        learning_rate = learning_rate,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 7,
        optim = optim,
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = output_dir,
        report_to = "none", # Use "wandb" for Weights & Biases
    ),
)

# @markdown ### Response Only Training / 回答のみ学習
# @markdown If `True`, the model ignores user instructions and only trains on the assistant's response. Generally improves instruction following.
# @markdown `True`の場合、ユーザーの指示部分は無視し、アシスタントの回答部分のみを学習します。一般的に指示従順性が向上します。
train_response_only = True # @param {type:"boolean"}

if train_response_only:
    trainer = train_on_responses_only(
        trainer,
        instruction_part = "<|im_start|>user\n",
        response_part = "<|im_start|>assistant\n",
    )

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


Map (num_proc=6):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# @title Start Training / 学習開始
# @markdown Shows memory stats before and after training.
# @markdown 学習前後のメモリ統計を表示し、学習を実行します。

gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

trainer_stats = trainer.train()

used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

GPU = Tesla T4. Max memory = 14.741 GB.
3.826 GB of memory reserved.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 63
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 33,030,144 of 4,055,498,240 (0.81% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
7,0.196400
14,0.032500
21,0.010700
28,0.007900
35,0.004300
42,0.002600
49,0.002700
56,0.001400
63,0.002300


4112.908 seconds used for training.
68.55 minutes used for training.
Peak reserved memory = 8.961 GB.
Peak reserved memory for training = 5.135 GB.
Peak reserved memory % of max memory = 60.79 %.
Peak reserved memory for training % of max memory = 34.835 %.


In [ ]:
# @title Save Model / モデルの保存
# @markdown ### Saving Settings / 保存設定

# @markdown `save_method`: Choose where to save.
# @markdown `save_method`: 保存先を選択してください。
# @markdown - `local`: Save to Colab disk (lost after runtime resets). / Colabのディスクに保存（ランタイム終了で消えます）。
# @markdown - `push_to_hub`: Upload to Hugging Face Hub. / Hugging Face Hubにアップロードします。
save_method = "push_to_hub" # @param ["local", "push_to_hub"]

# @markdown `output_path`: Folder name (local) or "username/modelname" (Hub).
# @markdown `output_path`: 保存フォルダ名（localの場合）または "ユーザー名/モデル名"（Hubの場合）。
output_path = "rikunarita/Qwen3-4B-Thinking-2507-Genius-Mathematician" # @param {type:"string"}

# @markdown `HF_TOKEN`: Hugging Face Write Token (Required if pushing to hub). Read from Colab Secrets.
# @markdown `HF_TOKEN`: Hugging Faceの書き込みトークン（Hubにプッシュする場合に必須）。Colab Secretsから読み込みます。
import google.colab.userdata
HF_TOKEN = google.colab.userdata.get("HF_TOKEN")

if save_method == "local":
    model.save_pretrained(output_path)
    tokenizer.save_pretrained(output_path)
    print(f"Model saved locally to {output_path}")

elif save_method == "push_to_hub":
    if HF_TOKEN == "":
        print("Error: Please provide a Hugging Face token.")
    else:
        model.push_to_hub(output_path, token = HF_TOKEN)
        tokenizer.push_to_hub(output_path, token = HF_TOKEN)
        print(f"Model pushed to Hugging Face Hub: {output_path}")

README.md:   0%|          | 0.00/572 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   0%|          | 60.9kB /  132MB            

Saved model to https://huggingface.co/rikunarita/Qwen3-4B-Thinking-2507-Genius-Coder


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpoh7p2q6s/tokenizer.json: 100%|##########| 11.4MB / 11.4MB            

Model pushed to Hugging Face Hub: rikunarita/Qwen3-4B-Thinking-2507-Genius-Coder


# Integration of LoRA Adapter and Base Model / LoRAアダプタとベースモデルの結合
If you wish to combine the LoRA adapter with the base model, you can do so via the link below./LoRAアダプタとベースモデルを結合させたい方は以下のリンクから実行できます。

https://colab.research.google.com/drive/1GvZP-TxRe1wT_7ZJsYn93MVo5jv_TG5f?usp=sharing